# Object Oriented Programming
Adapted from a [notebook](https://github.com/yoavram/Py4Eng/blob/master/sessions/oop.ipynb) by Yoav Ram.

# Introduction to OOP


*Everything in Python is an object* so everything in Python has a class. 
You can find out which class an object belongs to by using the default property `__class__` or giving it to the function `type`. Even the return of the `type` function has a class - it's of class `type`!

In [ ]:
print(type(5))
print(type(type(5)))

In [ ]:
a = 'Hi'
print(type(a))
print(type(type(a)))

In [ ]:
type(type(type(type(type(a)))))

The basic object type is `object`:

In [ ]:
print(object)
print(type(object))

Every class implicitly inherits from `object`, so all classes have some default attributes:

In [ ]:
print(dir(object))
type(dir(object))

In [ ]:
object_dir = set(dir(object))
int_dir = set(dir(int))

print('Attributes in object but not in int:\n', object_dir - int_dir, '\n')
print('Attributes in int but not in object:\n', int_dir - object_dir, '\n')

Those methods that `int` has but `object` doesn't have define the behaviour of `int`.
Specificially, all the `__xxx__` methods are special methods that interact with Python in specific ways, defining operators etc.

## Class definition

In [ ]:
class MyClass:
    pass

We defined a new class called `MyClass`.
We can now create a new instance of our new class:

In [ ]:
my_class1 = MyClass()
my_class2 = MyClass()
print(id(my_class1))
print(type(my_class1))
print(id(my_class2))
print(type(my_class2))

Note that both instances have the same type but a different `id`, so they are not the same object:

In [ ]:
my_class1 is my_class2

We can check if an object is an instance of a class with the `isinstance` function:

In [ ]:
print(isinstance(my_class1, MyClass))
print(isinstance(my_class1, object))
print(isinstance(my_class1, int))

Because all classes have a `__doc__` attribute inherited from `object`, 
you can pass any instance to `help`:

In [ ]:
help(my_class1) # uses the __doc__ attribute

In [ ]:
type(my_class1)

In [ ]:
my_class1.__class__


# `Point` class

Let's learn more by writing a class for a point in a n-dimensional Euclidian space ($\mathbb{R}^n$).

We start with the class definition (`class`) and the constructor (`__init__`) which initialized the attributes of the class instance. This is a 2D point, but soon we'll change it to nD point.

Note that the first argument to methods (member functions) is always `self`, a reference to the instance.

We don't need to check the types of the `__init__` arguments; rather, we try to convert them to `float`, and let the conversion fail if the types are wrong.

In [ ]:
import numbers
import math

In [ ]:
class Point:
    """A point in 2D Euclidian plane.
    """
    
    def __init__(self, x, y):        
        # assert isinstance(x, (int, float)) and isinstance(y, (int, float))
        self.x = float(x)
        self.y = float(y)
        

In [ ]:
origin = Point(0, 0)
print('origin', origin.x, origin.y)

p = Point(1, 2)
print('p', p.x, p.y)


In [ ]:
print(p.__dict__)

Notice that when we send a `Point` to the console we get:

In [ ]:
p

Which is not useful, so we will define how `Point` is represented in the console by implementing `__repr__` and/or `__str__` which must return a string:

In [ ]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return f'{type(self).__name__}({self.x!r}, {self.y!r})'
        # !r calls repr() on the argument
        
    def __str__(self):
        # what a user wants to see
        return f'({self.x}, {self.y})'
    

In [ ]:
p = Point(1,2)
p

In [ ]:
print(p)

In [ ]:
print(f'{p!r}')

Let's modify this to an n-dimensional point. Also, it would also be nice to be able to do `x, y = p` or `x, y, z = p` rather than `x, y = p.x, p.y`. To do that, we implement the `__iter__` method, which should either return an iterator or be a generator function. We'll also re-implement `__init__` to accept an arbitrary number of coordinates, and `__repr__` to use the iterator:

In [ ]:
class Point:
    """A point in a Euclidian space.
    """
    
    def __init__(self, *coords):
        self.coords = tuple(float(x) for x in coords) # convert inputs to a tuple of floats
    
    def __iter__(self):
        return iter(self.coords) # delegate to tuple: return the tuple iterator

    def __len__(self):
        return len(self.coords) # delegate to tuple
     
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        formatted_coords = (f'{x!r}' for x in self) # format each coordinate with its own __repr__ method
        formatted_coords = ', '.join(formatted_coords) # join (reduce) all coordinates      
        cls_name = type(self).__name__ # the name of this class - we get it dynamically so that it will work with inheritance
        return f'{cls_name}({formatted_coords})'

In [ ]:
p = Point(5, 7.2)
print(p)
p

In [ ]:
for c in p:
    print(c)

In [ ]:
x, y = p
print(x, y, p)
len(p)

This is very nice because it can now work with an arbitrary number of coordinates:

In [ ]:
p = Point(1.5, 5.6, 23.3, 555, 2)
print(f'{p} is in ℝ^{len(p)}')

Next up we define a method to add two points. Addition is by elements, for a 2D point: $(x_1, y_1) + (x_2, y_2) = (x_1+x_2, y_1+y_2)$.

We also allow to add an `int` or `float`, in which case we add the point to another point with all coordinates equal to the argument value.

In [ ]:
class Point:
    """A point in a Euclidian space.
    """
    
    def __init__(self, *coords):        
        self.coords = tuple(float(x) for x in coords)
    
    def __iter__(self):
        return iter(self.coords)
    
    def __len__(self):
        return len(self.coords)
     
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        formatted_coords = (f'{x!r}' for x in self) # format each coordinate with its own __repr__ method
        formatted_coords = ', '.join(formatted_coords) # join (reduce) all coordinates      
        cls_name = type(self).__name__ # the name of this class - we get it dynamically so that it will work with inheritance
        return f'{cls_name}({formatted_coords})'
    
    def add(self, other):
        cls = type(self)
        if isinstance(other, numbers.Real):
            coords = (x + other for x in self)
            return cls(*coords)
        if isinstance(other, cls):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            coords = (x1 + x2 for x1, x2 in zip(self, other))
            return cls(*coords)
        raise TypeError(f"unsupported operand type(s) for +: '{cls.__name__}' and '{type(other).__name__}'")

In [ ]:
print(Point(1,1).add(Point(2,2)))

print(Point.add(Point(1,1), Point(2,2)))

In [ ]:
Point(1, 1, 1).add(2)

In [ ]:
Point(1,1).add('2')

A nicer way to implement addition is to **overload** the addition operator `+` by implementing `__add__`, which is a name Python reserves for addition operations (those are double underscores). In that case, we don't need to raise the `TypeError` exception but rather just return the `NotImplemented` constant and Python will take care of the error (after checking if the right hand object knows how to do right-add using `__radd__`):

In [ ]:
class Point:
    """A point in a Euclidian space.
    """
    
    def __init__(self, *coords):        
        self.coords = tuple(float(x) for x in coords)
    
    def __iter__(self):
        return iter(self.coords)
    
    def __len__(self):
        return len(self.coords)
     
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        formatted_coords = (f'{x!r}' for x in self) # format each coordinate with its own __repr__ method
        formatted_coords = ', '.join(formatted_coords) # join (reduce) all coordinates      
        cls_name = type(self).__name__ # the name of this class - we get it dynamically so that it will work with inheritance
        return f'{cls_name}({formatted_coords})'
    
    def __add__(self, other):
        cls = type(self)
        if isinstance(other, numbers.Real):
            coords = (x + other for x in self)
            return cls(*coords)
        if isinstance(other, cls):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            coords = (x1 + x2 for x1, x2 in zip(self, other))
            return cls(*coords)
        return NotImplemented

In [ ]:
Point(1,1) + Point(2,2)

In [ ]:
Point(1,1).__add__(Point(2, 2))

In [ ]:
Point.__add__(Point(1,1), Point(2, 2))

In [ ]:
Point(1, 1, 1) + 2

In [ ]:
Point(1, 1) + '2'

We want to be a able to compare `Point`s:

In [ ]:
Point(1,2) == Point(2,1)

In [ ]:
p1 = Point(1,2)
p2 = Point(1,2)
p1 == p2

In [ ]:
p = Point(0, 0)
p2 = p
p == p2

In [ ]:
Point(1,2) > Point(2,1)

In [ ]:
print(dir(object))

In [ ]:
o1 = object()
o2 = object()
o1.__gt__(o2)

So the default `==` checks by identity and `>` is not defined. 

We should overload both these operators by implementing `__eq__` and `__gt__`:

In [ ]:
# from functools import total_ordering

# @total_ordering
class Point:
    """A point in a Euclidian space.
    """
    
    def __init__(self, *coords):        
        self.coords = tuple(float(x) for x in coords)
    
    def __iter__(self):
        return iter(self.coords)
    
    def __len__(self):
        return len(self.coords)
     
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        formatted_coords = (f'{x!r}' for x in self) # format each coordinate with its own __repr__ method
        formatted_coords = ', '.join(formatted_coords) # join (reduce) all coordinates      
        cls_name = type(self).__name__ # the name of this class - we get it dynamically so that it will work with inheritance
        return f'{cls_name}({formatted_coords})'
    
    def __add__(self, other):
        cls = type(self)
        if isinstance(other, numbers.Real):
            return cls(*(x + other for x in self))
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            return cls(*(x1 + x2 for x1, x2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        if len(self) != len(other):
            raise ValueError('Unmatching number of coordinates') # Getting repetitive... How would you refactor this?
        comparisons = (z1 > z2 for z1, z2 in zip(self, other))
        return all(comparisons)    

First we check if two points are equal:

In [ ]:
Point(1,0) == Point(1,2)

In [ ]:
Point(1,0,0,1) == Point(1,0,0,1)

Then if one is *strictly* smaller than the other:

In [ ]:
Point(2,0) > Point(1,2)

In [ ]:
Point(5,6) > Point(1,2)

Note that by implementing `==` and `>` we have logically defined other comparison operators:

In [ ]:
Point(5,6) < Point(1,4)

In [ ]:
Point(5,6) <= Point(1,2)

The addition operator `+` returns a **new instance**.

In [ ]:
p = Point(1, 1, 4)
p2 = p
p += Point(1, 2, 0)
print(p)
print(p2)

Next we will write a method that implements `+=` efficiently, so that instead of returning a new instance, it changes the current instance **in-place**. This is called an [augmented arithmetic assignments](https://docs.python.org/3.5/reference/datamodel.html?highlight=__iadd__#object.__iadd__). These methods should return their result.

In [ ]:
class Point:
    """A point in a Euclidian space.
    """
    
    def __init__(self, *coords):        
        self.coords = tuple(float(x) for x in coords)
    
    def __iter__(self):
        return iter(self.coords)
    
    def __len__(self):
        return len(self.coords)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        formatted_coords = (f'{x!r}' for x in self) # format each coordinate with its own __repr__ method
        formatted_coords = ', '.join(formatted_coords) # join (reduce) all coordinates      
        cls_name = type(self).__name__ # the name of this class - we get it dynamically so that it will work with inheritance
        return f'{cls_name}({formatted_coords}) id:{id(self)}'
    
    def __add__(self, other):
        cls = type(self)
        if isinstance(other, numbers.Real):
            return cls(*(x + other for x in self))
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            return cls(*(x1 + x2 for x1, x2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        if len(self) != len(other):
            raise ValueError('Unmatching number of coordinates')
        return all(z1 > z2 for z1, z2 in zip(self, other))    
    
    def __iadd__(self, other):
        if isinstance(other, numbers.Real):
            self.coords = tuple(x + other for x in self)
            return self
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            self.coords = tuple(x1 + x2 for x1, x2 in zip(self, other))
            return self
        return NotImplemented

In [ ]:
p = Point(1, 1)
p2 = p
print(repr(p))
print(repr(p2))
p += Point(1,2)
print(repr(p))
print(repr(p2))

Let's write a method that given many points, checks if the current point is more extreme than the other points.

**Note:** specifying a function argument with a `*` before its name says that we can give zero or more values and they will be packed in a `tuple`.

In [ ]:
class Point:
    """A point in a Euclidian space.
    """
    
    def __init__(self, *coords):        
        self.coords = tuple(float(x) for x in coords)
    
    def __iter__(self):
        return iter(self.coords)
    
    def __len__(self):
        return len(self.coords)
     
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        formatted_coords = (f'{x!r}' for x in self) # format each coordinate with its own __repr__ method
        formatted_coords = ', '.join(formatted_coords) # join (reduce) all coordinates      
        cls_name = type(self).__name__ # the name of this class - we get it dynamically so that it will work with inheritance
        return f'{cls_name}({formatted_coords})'
    
    def __add__(self, other):
        cls = type(self)
        if isinstance(other, numbers.Real):
            return cls(*(x + other for x in self))
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            return cls(*(x1 + x2 for x1, x2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        if len(self) != len(other):
            raise ValueError('Unmatching number of coordinates')
        return all(z1 > z2 for z1, z2 in zip(self, other))    
    
    def __iadd__(self, other):
        if isinstance(other, numbers.Real):
            self.coords = tuple(x + other for x in self)
            return self
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            self.coords = tuple(x1 + x2 for x1, x2 in zip(self, other))
            return self
        return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)

In [ ]:
p = Point(5, 6)
p.is_extreme(Point(1,1))

In [ ]:
p.is_extreme(Point(1,1), Point(2,4), Point(4,2))

We can also use the method via the class instead of the instance, and give the instance of interest (the one that we want to know if it is the extreme) as the first argument `self`. Similarly, we can either do `'hi'.upper()` or `str.upper('hi')`.

In [ ]:
Point.is_extreme(Point(7,8), Point(1,1), Point(4,5), Point(2,3))

#Point(7,8).is_extreme(Point(1,1), Point(4,5), Point(2,3))

Let's add a method to return a polar representation of `Point`. We'll need to calculate the magnitude, which we might as well do with the `__abs__` method (corresponding to `abs`) and an `angle` method:

In [ ]:
i = -5
abs(i)

In [ ]:
abs(Point(1, 3))

In [ ]:
class Point:
    """A point in a Euclidian space.
    """
    
    def __init__(self, *coords):        
        self.coords = tuple(float(x) for x in coords)
    
    def __iter__(self):
        return iter(self.coords)
    
    def __len__(self):
        return len(self.coords)
     
    def __str__(self):
        return f"({', '.join(str(x) for x in self)})"
    
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        formatted_coords = (f'{x!r}' for x in self) # format each coordinate with its own __repr__ method
        formatted_coords = ', '.join(formatted_coords) # join (reduce) all coordinates      
        cls_name = type(self).__name__ # the name of this class - we get it dynamically so that it will work with inheritance
        return f'{cls_name}({formatted_coords})'
    
    def __add__(self, other):
        cls = type(self)
        if isinstance(other, numbers.Real):
            return cls(*(x + other for x in self))
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            return cls(*(x1 + x2 for x1, x2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        if len(self) != len(other):
            raise ValueError('Unmatching number of coordinates')
        return all(z1 > z2 for z1, z2 in zip(self, other))    
    
    def __iadd__(self, other):
        if isinstance(other, numbers.Real):
            self.coords = tuple(x + other for x in self)
            return self
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            self.coords = tuple(x1 + x2 for x1, x2 in zip(self, other))
            return self
        return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        return math.hypot(*self)
    
    def angle(self):
        if len(self) == 2:
            x, y = self
            return math.atan2(y, x)
        raise NotImplementedError('angle is only implemented for 2D points')

    def to_polar(self):
        if len(self) == 2:
            return abs(self), self.angle()
        raise NotImplementedError('to_polar is only implemented for 2D points')

In [ ]:
for p in (Point(1, 0), Point(-1, 0), Point(1, 1), Point(2, 2)):
    distance, angle = p.to_polar()
    print(f'Cartesian: {p}\tPolar: ({distance:.3f}, {angle:.3f})')

If we have `topolar` that returns a tuple of polar coordinates, we might want to have `frompolar` that creates a new instance from a tuple of polar coordinates. This is called an **alternative constructor** and must be implemented as a **class method** as we don't have an instance at the time we call the method:

In [ ]:
class Point:
    """A point in a Euclidian space.
    """
    
    def __init__(self, *coords):
        self.coords = tuple(float(x) for x in coords)
    
    def __iter__(self):
        return iter(self.coords)
    
    def __len__(self):
        return len(self.coords)
     
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        formatted_coords = (f'{x!r}' for x in self) # format each coordinate with its own __repr__ method
        formatted_coords = ', '.join(formatted_coords) # join (reduce) all coordinates      
        cls_name = type(self).__name__ # the name of this class - we get it dynamically so that it will work with inheritance
        return f'{cls_name}({formatted_coords})'
    
    def __add__(self, other):
        cls = type(self)
        if isinstance(other, numbers.Real):
            return cls(*(x + other for x in self))
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            return cls(*(x1 + x2 for x1, x2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        if len(self) != len(other):
            raise ValueError('Unmatching number of coordinates')
        return all(z1 > z2 for z1, z2 in zip(self, other))    
    
    def __iadd__(self, other):
        if isinstance(other, numbers.Real):
            self.coords = tuple(x + other for x in self)
            return self
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            self.coords = tuple(x1 + x2 for x1, x2 in zip(self, other))
            return self
        return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        if len(self) == 2:
            return math.hypot(*self) # optimized for 2D
        return math.sqrt(sum(x*x for x in self))
    
    def angle(self):
        if len(self) == 2:
            x, y = self
            return math.atan2(y, x)
        raise NotImplementedError('angle is only implemented for 2D points')

    def to_polar(self):
        if len(self) == 2:
            return abs(self), self.angle()
        raise NotImplementedError('to_polar is only implemented for 2D points')
    
    def __format__(self, spec):
        formatted_coords = (f'{x:{spec}}' for x in self) # format each coordinate with its own __repr__ method
        formatted_coords = ', '.join(formatted_coords) # join (reduce) all coordinates      
        cls_name = type(self).__name__ # the name of this class - we get it dynamically so that it will work with inheritance
        return f'{cls_name}({formatted_coords})'
        # How would you get rid of the code duplication?...
    
    @classmethod
    def from_polar(cls, radius, angle):
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        return cls(x, y)
        
    @staticmethod
    def some_static(a, b):
        print(a, b)

In [ ]:
print(f'{Point.from_polar(1, math.pi)=:.3f}')

In [ ]:
p1 = Point(1, math.pi)
p2 = Point.from_polar(*p1.to_polar())
p1, p2, p1 == p2

We got a `False` because there is some numerical error.

In [ ]:
Point.some_static(1, 2)

In [ ]:
p = Point(0, 0)
p.some_static(1, 2)

In [ ]:
Point.from_polar(1, 2)
Point.to_polar()

We might want to make our `Point` immutable. This is easy enough, we just need to change our `coords` to `__coords` to convey that it should not be used directly, and add `coords` property. We shouldn't implement `__iadd__`, though, in this case.

In [ ]:
class Point:
    """A point in a Euclidian space.
    """
    
    def __init__(self, *coords):
        # Notice the __ before coords
        self.__coords = tuple(float(x) for x in coords)
    
    # If p is a Point, p.coords will invoke the getter defined by the @property decorator.
    # We can also define a setter (@coords.setter) and a deleter (@coords.deleter).
    # p.coords = value will invoke the setter, and del p.coords the deleter.
    @property
    def coords(self):
        return self.__coords
    
    def __iter__(self):
        return iter(self.coords)
    
    def __len__(self):
        return len(self.coords)
     
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        formatted_coords = (f'{x!r}' for x in self) # format each coordinate with its own __repr__ method
        formatted_coords = ', '.join(formatted_coords) # join (reduce) all coordinates      
        cls_name = type(self).__name__ # the name of this class - we get it dynamically so that it will work with inheritance
        return f'{cls_name}({formatted_coords})'
    
    def __add__(self, other):
        cls = type(self)
        if isinstance(other, numbers.Real):
            return cls(*(x + other for x in self))
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            return cls(*(x1 + x2 for x1, x2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        if len(self) != len(other):
            raise ValueError('Unmatching number of coordinates')
        return all(z1 > z2 for z1, z2 in zip(self, other))    
    
#     def __iadd__(self, other):
#         if isinstance(other, numbers.Real):
#             self.__coords = tuple(x + other for x in self)
#             return self
#         if isinstance(other, type(self)):
#             if len(self) != len(other):
#                 raise ValueError('Unmatching number of coordinates')
#             self.__coords = tuple(x1 + x2 for x1, x2 in zip(self, other))
#             return self
#         return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        if len(self) == 2:
            return math.hypot(*self) # optimized for 2D
        return math.sqrt(sum(x*x for x in self))
    
    def angle(self):
        if len(self) == 2:
            x, y = self
            return math.atan2(y, x)
        raise NotImplementedError('angle is only implemented for 2D points')

    def to_polar(self):
        if len(self) == 2:
            return abs(self), self.angle()
        raise NotImplementedError('to_polar is only implemented for 2D points')

    # Our point can be converted into a complex number
    def __complex__(self):
        if len(self) == 2:
            return complex(*self)
        raise NotImplementedError('conversion to is only implemented only for 2D point')
        
    @classmethod
    def from_polar(cls, radius, angle):
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        return cls(x, y)
        
    # def __hash__(self):
    #     return hash(repr(self))

In [ ]:
p = Point(1, 1)
print(p)
p.coords

Note that we didn't have to change any other method (except remove `__iadd__`) thanks to the `@property` decorator.

However, we cannot directly change `coords`, making our class immutable, at least by convention:

In [ ]:
p.coords = (2, 3)

In [ ]:
print(p)
p._Point__coords = (2.0, 3.0)
print(p)

In [ ]:
d = {p: "some_value"}
print(d)

We can also define the `__getitem__` method so that we can get specific coordinates (we'll not define the `__setitem__` to keep the class "immutable", but of course this can be done).

By having `__len__` and `__getitem__` we are implementing the *sequence protocol*, and therefore we get `__iter__` and `__reversed__` for free, so we can throw `__iter__` away (although we might want to keep it if we think our implementation is better).

We can also define `x`, `y`, and `z` properties as aliases for the first 3 coordinates using properties.

In [ ]:
class Point:
    """A point in a Euclidian space.
    """
    
    def __init__(self, *coords):        
        self.__coords = tuple(float(x) for x in coords)
    
    @property
    def coords(self):
        return self.__coords

    def __getitem__(self, key):
        return self.coords[key]

    @property
    def x(self):
        return self[0]

    @property
    def y(self):
        return self[1]
    
    @property
    def z(self):
        return self[2]

    def __len__(self):
        return len(self.coords)
     
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        formatted_coords = (f'{x!r}' for x in self) # format each coordinate with its own __repr__ method
        formatted_coords = ', '.join(formatted_coords) # join (reduce) all coordinates      
        cls_name = type(self).__name__ # the name of this class - we get it dynamically so that it will work with inheritance
        return f'{cls_name}({formatted_coords})'
    
    def __add__(self, other):
        cls = type(self)
        if isinstance(other, numbers.Real):
            return cls(*(x + other for x in self))
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            return cls(*(x1 + x2 for x1, x2 in zip(self, other)))
        return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        if len(self) != len(other):
            raise ValueError('Unmatching number of coordinates')
        return all(z1 > z2 for z1, z2 in zip(self, other))    
    
    def __iadd__(self, other):
        if isinstance(other, numbers.Real):
            self._coords = tuple(x + other for x in self)
            return self
        if isinstance(other, type(self)):
            if len(self) != len(other):
                raise ValueError('Unmatching number of coordinates')
            self._coords = tuple(x1 + x2 for x1, x2 in zip(self, other))
            return self
        return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        if len(self) == 2:
            return math.hypot(*self) # optimized for 2D
        return math.sqrt(sum(x*x for x in self))
    
    def angle(self):
        if len(self) == 2:
            x, y = self
            return math.atan2(y, x)
        raise NotImplementedError('angle is only implemented for 2D points')

    def to_polar(self):
        if len(self) == 2:
            return abs(self), self.angle()
        raise NotImplementedError('to_polar is only implemented for 2D points')
    
    def __complex__(self):
        if len(self) == 2:
            return complex(*self)
        raise NotImplementedError('conversion to is only implemented only for 2D point')
        
    @classmethod
    def from_polar(cls, radius, angle):
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        return cls(x, y)

In [ ]:
p = Point(1, 2, 3, 4, 5)
p.coords

In [ ]:
p[0], p[1]

In [ ]:
p[-1]

In [ ]:
p[10]

In [ ]:
p[2:4]

In [ ]:
p.x, p.y

### `__dict__`

An object attributes are saved in a dictionary called `__dict__`:

In [ ]:
p.__dict__

However, a dictionary has some overhead, and a more memory efficient way to save the attributes is done by defining the class attribute `__slots__`.

Let's see an example for 2D point:

In [ ]:
class RegularPoint:  
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return f'{type(self).__name__}({self.x}, {self.y})'
    
class EfficientPoint:
    __slots__ = ('x', 'y')
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return f'{type(self).__name__}({self.x}, {self.y})'

In [ ]:
p_reg = RegularPoint(0, 0)
p_eff = EfficientPoint(0, 0)

In [ ]:
print(p_reg.__dict__)
print(p_eff.__dict__)

In [ ]:
p_reg.z = 10
print(p_reg.z)

In [ ]:
p_eff.z = 10

We load [ipython_memory_usage](https://github.com/ianozsvald/ipython_memory_usage), a package that monitors memory usage inside the notebook.

In [ ]:
%pip install ipython_memory_usage
%pip install psutil

In [ ]:
import ipython_memory_usage.ipython_memory_usage as imu
imu.start_watching_memory()

In [ ]:
lst1 = [RegularPoint(0, 0) for _ in range(10_000_000)]

In [ ]:
lst2 = [EfficientPoint(0, 0) for _ in range(10_000_000)]

So we see that for $10^7$ points, the `__slots__` class requires ~2-fold less memory.

We should now delete the references to release these GB of RAM with the `del` statement:

In [ ]:
del lst1
del lst2

And finally stop monitoring the memory:

In [ ]:
imu.stop_watching_memory()

## `Rectangle` class

We will implement two classes for rectangles, and compare the two implementations.

### First implementation - two points

The first implementation defines a rectangle by its lower left and upper right vertices.

In [ ]:
class Rectangle1:
    """Describe a parallel-axes rectangle by storing two points.
    
    Attributes
    ----------
    llv : Point
        lower left vertex
    urv : Point
        upper right vertex
    """
    
    def __init__(self, lower_left_vertex, upper_right_vertex):        
        self.llv = Point(*lower_left_vertex)
        self.urv = Point(*upper_right_vertex)
        assert self.llv < self.urv, "Lower left vertex should be lower than upper right vertex"
        
    def __repr__(self):
        return f"{type(self).__name__}({self.llv!r}, {self.urv!r})"
        
    def __str__(self):
        return f"Rectangle with lower left {self.llv} and upper right {self.urv}"

    def dimensions(self):
        height = self.urv.y - self.llv.y
        width = self.urv.x - self.llv.x
        return height, width
    
    def area(self):
        height, width = self.dimensions()
        area = height * width
        return area
       
    def transpose(self):
        """Reflection with regard to the line passing through lower left vertex with angle 315 (-45) degrees
        """
        height, width = self.dimensions()
        urv = self.llv
        llv = Point(self.llv.x - height, self.llv.y - width)
        return type(self)(llv, urv)

In [ ]:
rect = Rectangle1(Point(0, 0), Point(2, 1))
print(rect)
print("Area:", rect.area())
print("Dimensions:", rect.dimensions())
t_rect = rect.transpose()
print("Transposed:", t_rect)

## Exercise

Overload the `__contains__` method of the rectangle class so that given a point it will return `True` if the point is inside the rectangle and `False` otherwise.

In [ ]:
rect = Rectangle1(Point(0, 0), Point(1, 2))
assert Point(0.5, 0.5) in rect
assert Point(1, 2) not in rect
assert Point(0, 2) not in rect
assert Point(-0.5, 0.5) not in rect
assert Point(2.5, 0.5) not in rect
assert Point(0.5, 2.5) not in rect

## Exercise

The second implementation defines a rectangle by the lower left point, the height and the width.

Define the exact same methods as in `Rectangle1`, with the same input and output, but  different inner representation / implementation.

In [ ]:
class Rectangle2:
    """Describe a parallel-axes rectangle by storing lower left point, height and width.
    
    Attributes
    ----------
    point : Point
        lower left point
    width : float
        width
    height : float
        height
    """
    pass

In [ ]:
rect = Rectangle2(Point(0, 0), 1, 2)
print(rect)
print("Area:", rect.area())
print("Dimensions:", rect.dimensions())
t_rect = rect.transpose()
print("Transposed:", t_rect)

# Inheritance

Next we will see how inheritance works in Python.

We define a `Door` class. The door is either open or closed, it can be opened or closed, and it can be represented as a string.

In [ ]:
class Door:
    OPEN = 'open'
    CLOSED = 'closed'
    
    def __init__(self, status=CLOSED):
        self.__status = status
        
    @property
    def status(self):
        return self.__status

    @status.setter
    def status(self, new_status):
        if new_status not in (self.OPEN, self.CLOSED):
            raise ValueError(f'Status must be one of {(self.OPEN, self.CLOSED)}')
        self.__status = new_status
        return new_status
    
    def __repr__(self):
        return f'{type(self).__name__}(status={self.status})'

    def __str__(self):
        return f'Door {id(self)} is {self.status}'
    
    def open(self):
        self.status = self.OPEN
        
    def close(self):
        self.status = self.CLOSED

In [ ]:
door = Door()
print(door)
door

Now we want to define a secure door which only opens with a password.

The secure door inherits from the regular door, but it makes some changes:

- `__init__` accepts a password and saves it's hash as an attribute
- `open` requires the password from the user to actually open the door

The `super` function gives access to the parent class.

In [ ]:
from getpass import getpass
from hashlib import sha224

def digest(password):
    """Hash a password using sha224 algorithm.
    """
    return sha224(password.encode('utf8')).hexdigest()

In [ ]:
class SecurityDoor(Door):
    """A door that requires a password to open.
    """
  
    def __init__(self, password):
        super().__init__()
        self.secret = digest(password)
        
    def open(self):
        if digest(getpass('What is the password?')) == self.secret:
            #super(SecurityDoor, self).open()
            super().open()
        else:
            print('Wrong password!')

In [ ]:
secure_door = SecurityDoor('opensesame')
secure_door.open()
print(secure_door)
secure_door.close()
print(secure_door)

# Method Resolution Order

Remember the diamond problem in multiple inheritance?
Let's see what happens in python.

<div>
<img src="images/Diamond_inheritance.svg" width="200"/>
</div>

In [ ]:
class a:
    value = 'a'
        
class b(a):
    value = 'b'    

class c(a):
    value = 'c'    
    
class d(b, c):
    pass
    
    
d_obj = d()
print(d_obj.value)

In [ ]:
print(d.__mro__)

Multiple inheritance works in Python. If there are clashes between names Python will search from the first parent to the last. 

However, multiple inheritance and inheritance in general are many times not neccessary and similar results can be produced by composition and delegation, without creating the complexity and constraints that inheritance induces.

In the words of the famous [Design Patterns](https://en.wikipedia.org/wiki/Composition_over_inheritance) book:

> Favor 'object composition' over 'class inheritance'.

# Composition and delegation

Composition means that one object **explicitly delegates** some tasks to another object. 

Let's implement regular composition, which simply makes an object part of the other as an attribute:

In [ ]:
class SecurityDoor:
    """A door that requires a password to open.
    """
    
    def __init__(self, password):
        self.__door = Door(status=Door.CLOSED)
        self.__secret = digest(password)
    
    @property
    def secret(self):
        return self.__secret
    
    @property
    def door(self):
        return self.__door
    
    def __repr__(self):
        return repr(self.door)
               
    def open(self):
        if digest(getpass('What is the password?')) == self.secret:
            self.door.open()
        else:
            print('Wrong password!')
        
    def close(self):
        self.door.close()

In [ ]:
secure_door = SecurityDoor('opensesame')
secure_door.open()
print(secure_door)
secure_door.close()
print(secure_door)

In [ ]:
secure_door.OPEN



The primary goal of composition is to relax the coupling between objects. 
This little example shows that now `SecurityDoor` is an `object` and no more a `Door`, 
which means that the internal structure of `Door` is not copied. 
For this very simple example both `Door` and `SecurityDoor` are not big classes, 
but in a real system objects can very complex; 
this means that their allocation consumes a lot of memory and if a system contains thousands or millions of objects that could be an issue.

The composed `SecurityDoor` has to redefine every attribute since the concept of delegation applies only to methods and not to attributes, doesn't it?

**No.** Python allows objects manipulation and attribute access is one of the most useful. 
Accessing attributes is ruled by a special method called `__getattribute__` that is called whenever an attribute of the object is accessed. 
Overriding `__getattribute__`, however, is overkill; 
it is a very complex method, and, being called on every attribute access, any change makes the whole thing slower.

The method we have to leverage to delegate attribute access is `__getattr__`, which is a special method that is called whenever the requested attribute is not found in the object. 
So basically it is the right place to dispatch all attribute and method access our object cannot handle. 

The previous example becomes:

In [ ]:
class SecurityDoor:
    """A door that requires a password to open.
    """
    
    def __init__(self, password):
        self.__door = Door(status=Door.CLOSED)
        self.__secret = digest(password)
              
    @property
    def door(self):
        return self.__door
    
    @property
    def secret(self):
        return self.__secret
            
    def __repr__(self):
        return repr(self.door)
    
    def open(self):
        if digest(getpass('What is the password?')) == self.secret:
            self.door.open()
        else:
            print('Wrong password!')
        
    # def close(self):
    #     self.door.close()
        
    def __getattr__(self, attr):
        return getattr(self.door, attr)

In [ ]:
secure_door = SecurityDoor('opensesame')
secure_door.open()
print(secure_door)
secure_door.close()
print(secure_door)

In [ ]:
secure_door.OPEN

As this last example shows, delegating every member access through `__getattr__` is very simple. 
Pay attention to `getattr` which is different from `__getattr__`. 
The former is a built-in function that is equivalent to the dotted syntax, i.e. `getattr(obj, 'someattr')` is the same as `obj.someattr`, but you have to use it since the name of the attribute is contained in a string.

**Composition provides a superior way to manage delegation** since it can selectively delegate the access, even mask some attributes or methods, while inheritance cannot. 

In Python you also avoid the memory problems that might arise with delegation when you put many objects inside another; Python handles everything through its reference, so the size of an attribute is constant and very limited.

## Exercise

Define a new class, `RevolvingDoor`, which closes immediately after it is opened. Define it once with inheritance and once with composition.

In [ ]:
class RevolvingDoor(Door):
    pass

rdoor = RevolvingDoor()
rdoor.open()
print(rdoor)

In [ ]:
class RevolvingDoor:
    pass
    
rdoor = RevolvingDoor()
rdoor.open()
print(rdoor)

# Polymorphism

In Python, polymorphism is baked into the language, due to the **Duck typing** principle. We saw above how this relates to methods like `__add__`, `__sub__`, `__repr__`, and `__contains__`.

Another example is a file object:

In [ ]:
f = open("./data/crops.txt")
print(type(f))
print(dir(f))
f.close()

Say we write a function that gets a file and returns all the lines that start with a given prefix (say, 'Am'):

In [ ]:
def filter_by_prefix(file, prefix):
    return [line.strip() for line in file if line.startswith(prefix)]

with open('./data/crops.txt', 'r') as f:
    print(filter_by_prefix(f, 'Am'))

But now, say we want to read from `crops.txt.gz`, which is compressed with gzip. There is a module for reading gzipped files:

In [ ]:
import gzip

In [ ]:
gzfile = gzip.open('./data/crops.txt.gz' ,'r')
print(type(gzfile))
print(dir(gzfile))
print(isinstance(gzfile, f.__class__))
gzfile.close()

You notice that this is a different type than a file we opened with `open` (that would be `f`), and the is not even inheritance relationship (we know this because `isinstace` returned `False`). 

But our function `filter_by_prefix` doesn't care about the type, all it wants is an **object that you can loop over with a `for`**: an iterable object that implements the `__iter__` method by either:

- returning an iterator, which is an object that has a `next` method and stops iteration by raising a `StopIteration` exception, or 
- using the `yield` statement, which creates a generator.

Indeed, `gzfile` implements `__iter__`:

In [ ]:
hasattr(gzfile, '__iter__')

So we can use it with our function:

In [ ]:
with gzip.open('./data/crops.txt.gz', 'rt') as gzfile: # rt is for reading text
    print(filter_by_prefix(gzfile, 'Am'))

It doesn't even have to be a file. A list is just as good:

In [ ]:
continents = ['America', 'Europe', 'Asia', 'Africa', 'Anarctica', 'Australia']
print(filter_by_prefix(continents, 'Am'))

# Destructor

Python uses a garbase collector, and the standard CPython implementation uses reference counting and deletes objects when there are no more references to them.

You can remove references using the `del` command (see in [Python Tutor](http://pythontutor.com/visualize.html#code=a+%3D+%5B1,2,3%5D%0Adel+a&mode=display&origin=opt-frontend.js&cumulative=false&heapPrimitives=false&textReferences=false&py=3&rawInputLstJSON=%5B%5D&curInstr=2)):

In [ ]:
a = [1,2,3]
del a
a

You can implement the [`__del__`](https://docs.python.org/3.5/reference/datamodel.html#object.__del__) method which is called when an instance is about to be destroyed. 
This is also called a **destructor**. 
Similar to `__init__`, if a base (parent) class has a `__del__` method then you must explicitly call it. 
It is not guaranteed that __del__() methods are called for objects that still exist when the interpreter exits.

In [ ]:
class A:
    def __init__(self, name):
        self.name = name
        print('Initialize')
        
    def __del__(self):
        print('Delete')
        
    def __repr__(self):
        return self.name

In [ ]:
a = A('Alex')
print(a)

In [ ]:
b = a
print(b)

In [ ]:
del a
print(b)

In [ ]:
del b

Note that several things can implicitly save a reference to your object, including the `_` variable name and stack traces.

# References

- Python 3 [OOP tutorial](https://docs.python.org/3/tutorial/classes.html)
- Leonardo Giordani's [OOP notebooks](http://nbviewer.jupyter.org/github/lgiordani/blog_source/blob/master/pelican/content/notebooks/Python_3_OOP_Part_3__Delegation__composition_and_inheritance.ipynb)
- [ Fluent Python](http://shop.oreilly.com/product/0636920032519.do) by Luciano Ramalho is a great book for intermediate and advanced Python programmers.
- [Python data model docs](https://docs.python.org/3.5/reference/datamodel.html).